In [1]:
import os
import math
import pandas as pd
import numpy as np
import urllib
import matplotlib.pyplot as plt
%matplotlib inline

import hydrofunctions as hf

import fbprophet 

# For inputting City information to get distance to sensors.
from  geopy.geocoders import Nominatim

# Date time conversion registration.
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()


In [2]:
site = '09081000'
start = '2000-01-23'
end = '2020-01-23'

herring = hf.NWIS(site, 'dv', start, end)

In [3]:
herring.get_data()

In [4]:
my_dict = herring.json()
df = hf.extract_nwis_df(my_dict)
df.columns

Index(['USGS:09081000:00060:00003', 'USGS:09081000:00060:00003_qualifiers'], dtype='object')

In [5]:
df.head()

,USGS:09081000:00060:00003,USGS:09081000:00060:00003_qualifiers
datetime,,
2000-01-23,252.0,A
2000-01-24,252.0,A
2000-01-25,264.0,A
2000-01-26,271.0,A
2000-01-27,262.0,A


In [19]:
df.plot()
plt.yscale('log')
plt.savefig('RRFork1.png')
plt.close()

In [8]:
df.rename(index=str, columns = {"USGS:09081000:00060:00003" : "y", 
                                "USGS:09081000:00060:00003_qualifiers" : "flags"}, 
         inplace = True)

df['ds'] = df.index[:]

df.head()

,y,flags,ds
datetime,,,
2000-01-23 00:00:00,252.0,A,2000-01-23 00:00:00
2000-01-24 00:00:00,252.0,A,2000-01-24 00:00:00
2000-01-25 00:00:00,264.0,A,2000-01-25 00:00:00
2000-01-26 00:00:00,271.0,A,2000-01-26 00:00:00
2000-01-27 00:00:00,262.0,A,2000-01-27 00:00:00


In [16]:
df_prophet = fbprophet.Prophet(changepoint_prior_scale=0.05, daily_seasonality=True, interval_width = 0.25)
df_prophet.fit(df)

In [17]:
df_forecast = df_prophet.make_future_dataframe(periods=365 * 1, freq='D')
df_forecast = df_prophet.predict(df_forecast)

In [18]:
df_prophet.plot(df_forecast, xlabel = 'Date', ylabel = 'Discharge (cfs)')
plt.title('Discharge at ROARING FORK RIVER NEAR EMMA, CO')
plt.ylim(55,)
plt.yscale('log')
plt.savefig('RRFork1_Pred.png')
plt.close()